In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bisect
from sklearn.preprocessing import *
import mpl_toolkits.mplot3d
from sklearn.cluster import KMeans
import ffn as ffn
import empyrical as ep
import ffn as ffn
from sktime.forecasting.model_selection import SlidingWindowSplitter
import talib


In [61]:
N_CLOSE_PTS = 24
N_PERC_PTS = 5
DIST_MEASURE = 1
FUTURE_CANDLES = 10
N_CLUSTERS = 100
LOG_RETURN_THRESHOLD = 0
CALMAR_RATIO_THRESHOLD = 2
ATR_MULTIPLIER = 10

INIT_CAPITAL = 100
RETURNS_CONSTANT = 2500

ONE_DAY = 4 * 24
first_train_size = 150 * ONE_DAY  # 150 days
second_train_size = 100 * ONE_DAY  # 100 days

train_size = first_train_size + second_train_size
test_size = 10 * ONE_DAY  # 10 days
val_test_horizon = test_size * 2
random_state = 42

In [58]:
ohlcv_data

,open,high,low,close,volume,year,month,day_of_week,hour,minute,log_close,log_high,log_low,log_atr
time,,,,,,,,,,,,,,
2007-01-01 21:30:00+00:00,1.95840,1.95860,1.95840,1.95840,18,2007,1,0,21,30,0.672128,0.672230,0.672128,0.000117
2007-01-01 21:45:00+00:00,1.95840,1.95965,1.95830,1.95920,47,2007,1,0,21,45,0.672536,0.672766,0.672077,0.000689
2007-01-01 22:00:00+00:00,1.95910,1.96005,1.95890,1.95978,60,2007,1,0,22,0,0.672832,0.672970,0.672383,0.000587
2007-01-01 22:15:00+00:00,1.96003,1.96190,1.96003,1.96156,91,2007,1,0,22,15,0.673740,0.673913,0.672960,0.001081
2007-01-01 22:30:00+00:00,1.96138,1.96174,1.96083,1.96174,82,2007,1,0,22,30,0.673832,0.673832,0.673368,0.000464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-01 22:45:00+00:00,1.55835,1.55842,1.55732,1.55794,118,2015,1,3,22,45,0.443364,0.443672,0.442966,0.000706
2015-01-01 23:00:00+00:00,1.55800,1.55841,1.55722,1.55828,214,2015,1,3,23,0,0.443583,0.443666,0.442902,0.000764
2015-01-01 23:15:00+00:00,1.55832,1.55832,1.55694,1.55704,589,2015,1,3,23,15,0.442787,0.443608,0.442722,0.000886


In [62]:
ohlcv_data = pd.read_csv(
    "/Users/newuser/Projects/robust_algo_trader/data/gen_oanda_data/GBP_USD_M15_raw_data.csv",
    parse_dates=["time"],
)
ohlcv_data = ohlcv_data.set_index("time")
ohlcv_data["year"] = ohlcv_data.index.year
ohlcv_data["month"] = ohlcv_data.index.month
ohlcv_data["day_of_week"] = ohlcv_data.index.dayofweek
ohlcv_data["hour"] = ohlcv_data.index.hour
ohlcv_data["minute"] = ohlcv_data.index.minute
ohlcv_data["log_close"] = np.log(ohlcv_data["close"])
ohlcv_data["log_high"] = np.log(ohlcv_data["high"])
ohlcv_data["log_low"] = np.log(ohlcv_data["low"])
ohlcv_data["log_atr"] = talib.ATR(ohlcv_data["log_high"], ohlcv_data["log_low"], ohlcv_data["log_close"], timeperiod=1)

start_date = "2007-01-01"
end_date = "2015-01-01"
ohlcv_data = ohlcv_data[start_date:end_date]
df = ohlcv_data.copy()
df

,open,high,low,close,volume,year,month,day_of_week,hour,minute,log_close,log_high,log_low,log_atr
time,,,,,,,,,,,,,,
2007-01-01 21:30:00+00:00,1.95840,1.95860,1.95840,1.95840,18,2007,1,0,21,30,0.672128,0.672230,0.672128,0.000117
2007-01-01 21:45:00+00:00,1.95840,1.95965,1.95830,1.95920,47,2007,1,0,21,45,0.672536,0.672766,0.672077,0.000689
2007-01-01 22:00:00+00:00,1.95910,1.96005,1.95890,1.95978,60,2007,1,0,22,0,0.672832,0.672970,0.672383,0.000587
2007-01-01 22:15:00+00:00,1.96003,1.96190,1.96003,1.96156,91,2007,1,0,22,15,0.673740,0.673913,0.672960,0.001081
2007-01-01 22:30:00+00:00,1.96138,1.96174,1.96083,1.96174,82,2007,1,0,22,30,0.673832,0.673832,0.673368,0.000464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-01-01 22:45:00+00:00,1.55835,1.55842,1.55732,1.55794,118,2015,1,3,22,45,0.443364,0.443672,0.442966,0.000706
2015-01-01 23:00:00+00:00,1.55800,1.55841,1.55722,1.55828,214,2015,1,3,23,0,0.443583,0.443666,0.442902,0.000764
2015-01-01 23:15:00+00:00,1.55832,1.55832,1.55694,1.55704,589,2015,1,3,23,15,0.442787,0.443608,0.442722,0.000886


In [ ]:
# Define a function to calculate the ulcer index
def m_ulcer_index(series):
    drawdown = (series - series.cummax()) / series.cummax()
    squared_average = (drawdown**2).mean()
    return squared_average**0.5


# DIST_MEASURE
# 1 = Euclidean Distance
# 2 = Perpendicular Distance
# 3 = Vertical Distance
def find_pips(data, n_pips):
    pips_x = [0, len(data) - 1]  # Index
    pips_y = [data[0], data[-1]]  # Price
    for curr_point in range(2, n_pips):
        md = 0.0  # Max distance
        md_i = -1  # Max distance index
        insert_index = -1
        # Use a single loop to iterate over all the points
        for i in range(1, len(data) - 1):
            left_adj = bisect.bisect_right(pips_x, i) - 1
            right_adj = left_adj + 1
            # Calculate the distance from the point to the line segment
            d = distance(data, pips_x, pips_y, i, left_adj, right_adj)
            # Update the maximum distance and the insert index
            if d > md:
                md = d
                md_i = i
                insert_index = right_adj
        # Insert the new pip
        pips_x.insert(insert_index, md_i)
        pips_y.insert(insert_index, data[md_i])
    return pips_x, pips_y


# Define a helper function to calculate the distance
def distance(data, pips_x, pips_y, i, left_adj, right_adj):
    time_diff = pips_x[right_adj] - pips_x[left_adj]
    price_diff = pips_y[right_adj] - pips_y[left_adj]
    slope = price_diff / time_diff
    intercept = pips_y[left_adj] - pips_x[left_adj] * slope
    dist_funcs = {
        1: lambda x, y: ((pips_x[left_adj] - x) ** 2 + (pips_y[left_adj] - y) ** 2)
        ** 0.5
        + ((pips_x[right_adj] - x) ** 2 + (pips_y[right_adj] - y) ** 2) ** 0.5,
        2: lambda x, y: abs((slope * x + intercept) - y) / (slope**2 + 1) ** 0.5,
        3: lambda x, y: abs((slope * x + intercept) - y),
    }
    return dist_funcs[DIST_MEASURE](i, data[i])


def get_pips_df(sub_df):
    pips_y_list = []
    # loop through the data
    for index in range(N_CLOSE_PTS, len(sub_df)):
        try:
            x_close = sub_df["log_close"].iloc[index - N_CLOSE_PTS : index].to_numpy()
            pips_x, pips_y = find_pips(x_close, N_PERC_PTS)
            scaled_pips_y = (
                StandardScaler()
                .fit_transform(np.array(pips_y).reshape(-1, 1))
                .reshape(-1)
            )
            pips_y_dict = {f"pip_{i}": scaled_pips_y[i] for i in range(N_PERC_PTS)}
            j = index - 1
            pips_y_dict["year"] = sub_df["year"].iloc[j]
            pips_y_dict["month"] = sub_df["month"].iloc[j]
            pips_y_dict["day_of_week"] = sub_df["day_of_week"].iloc[j]
            pips_y_dict["hour"] = sub_df["hour"].iloc[j]
            pips_y_dict["minute"] = sub_df["minute"].iloc[j]
            # future features
            tp = sub_df["log_close"].iloc[j] + (ATR_MULTIPLIER * sub_df["log_atr"].iloc[j])
            sl = sub_df["log_close"].iloc[j] - (ATR_MULTIPLIER * sub_df["log_atr"].iloc[j])
            # future_return should be 1 when tp_or_sl is hit and -1 if -tp_or_sl is hit
            # loop through future sub_df["log_atr"] until tp_or_sl or -tp_or_sl is hit
            for k in range(index, len(sub_df)):
                if sub_df["log_close"].iloc[k] >= tp:
                    pips_y_dict["future_return"] = 1
                    break
                elif sub_df["log_close"].iloc[k] <= sl:
                    pips_y_dict["future_return"] = -1
                    break
                else:
                    pips_y_dict["future_return"] = 0
            pips_y_list.append(pips_y_dict)
        except Exception as e:
            print(e)
            break
    return pd.DataFrame(pips_y_list)


def cluster_and_filter_pips_df(pips_y_df, pips_second_train_df):
    pips_y_df_np = pips_y_df[
        ["pip_0", "pip_1", "pip_2", "pip_3", "pip_4", "day_of_week", "hour", "minute"]
    ].to_numpy()
    kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=random_state)
    kmeans.fit(pips_y_df_np)
    pips_y_df["k_label"] = kmeans.labels_
    k_label_returns_df = (
        pips_y_df.groupby("k_label")["future_return"]
        .sum()
        .sort_values(ascending=False)
    )
    # print("k_label_returns_df")
    # print(k_label_returns_df)

    pips_second_train_df["k_label"] = kmeans.predict(
        pips_second_train_df[
            [
                "pip_0",
                "pip_1",
                "pip_2",
                "pip_3",
                "pip_4",
                "day_of_week",
                "hour",
                "minute",
            ]
        ].to_numpy()
    )

    best_k_labels_list = []
    for k_label in k_label_returns_df.index:
        pips_y_sub_df = pips_y_df[(pips_y_df["k_label"] == k_label)]
        k_label_cumsum = (
            pips_y_sub_df["future_return"].cumsum().reset_index(drop=True)
        )
        if k_label_cumsum.iloc[-1] > 0:
            signal = 1
        else:
            signal = 0
            k_label_cumsum = -k_label_cumsum

        print("k_label_cumsum")
        print(k_label_cumsum)
        # Add a constant value to the series
        # Put INIT_CAPITAL as the first value
        start_portfolio = pd.concat([pd.Series([INIT_CAPITAL]), 
                                     (k_label_cumsum + INIT_CAPITAL)]).reset_index(drop=True)
        if not start_portfolio.empty:
            start_k_label_cumsum = start_portfolio.iloc[0]
            end_k_label_cumsum = start_portfolio.iloc[-1]
        else:
            continue

        annualized_return = (end_k_label_cumsum / start_k_label_cumsum) - 1
        ulcer_index = m_ulcer_index(start_portfolio)
        max_drawdown = abs(ffn.calc_max_drawdown(start_portfolio)) + 0.001
        calmar_ratio = annualized_return / max_drawdown
        if (
            True
            and calmar_ratio > CALMAR_RATIO_THRESHOLD
            and annualized_return > LOG_RETURN_THRESHOLD
        ):

            pips_second_train_df = pips_second_train_df[
                pips_second_train_df["k_label"] == k_label
            ]
            k_label_cumsum = (
                pips_second_train_df["future_return"].cumsum().reset_index(drop=True)
            )
            if signal == 0:
                k_label_cumsum = -k_label_cumsum
                
            end_portfolio = pd.concat([pd.Series([INIT_CAPITAL]), 
                                     (k_label_cumsum + INIT_CAPITAL)]).reset_index(drop=True)
            
            if not end_portfolio.empty:
                start_k_label_cumsum = end_portfolio.iloc[0]
                end_k_label_cumsum = end_portfolio.iloc[-1]
            else:
                continue
            annualized_return = (end_k_label_cumsum / start_k_label_cumsum) - 1
            ulcer_index = m_ulcer_index(end_portfolio)
            max_drawdown = abs(ffn.calc_max_drawdown(end_portfolio)) + 0.001
            calmar_ratio = annualized_return / max_drawdown
            if (
                True
                and calmar_ratio > CALMAR_RATIO_THRESHOLD
                and annualized_return > LOG_RETURN_THRESHOLD
            ):
                best_k_labels_list.append(
                    {
                        "signal": signal,
                        "k_label": k_label,
                        "calmar_ratio": calmar_ratio,
                        "ulcer_index": ulcer_index,
                        "annualized_return": annualized_return,
                        "max_drawdown": max_drawdown,
                    }
                )
    best_k_labels_df = pd.DataFrame(best_k_labels_list)
    return best_k_labels_df, kmeans


def filter_pips_df(pips_y_df, train_best_k_labels_df, kmeans):
    pips_y_df_np = pips_y_df[
        ["pip_0", "pip_1", "pip_2", "pip_3", "pip_4", "day_of_week", "hour", "minute"]
    ].to_numpy()
    pips_y_df["k_label"] = kmeans.predict(pips_y_df_np)

    val_best_k_labels_list = []

    for i in range(len(train_best_k_labels_df)):
        k_label = train_best_k_labels_df.iloc[i]["k_label"]
        signal = train_best_k_labels_df.iloc[i]["signal"]
        pips_y_copy_df = pips_y_df[(pips_y_df["k_label"] == k_label)]
        val_k_label_cumsum = pips_y_copy_df["future_return"].cumsum()
        val_k_label_cumsum = val_k_label_cumsum.reset_index(drop=True)
        if signal == 0:
            val_k_label_cumsum = -val_k_label_cumsum
        # Add a constant value to the series
        val_portfolio = pd.concat([pd.Series([INIT_CAPITAL]), 
                                     (val_k_label_cumsum + INIT_CAPITAL)]).reset_index(drop=True)
        
        if not val_portfolio.empty:
            start_val_k_label_cumsum = val_portfolio.iloc[0]
            end_val_k_label_cumsum = val_portfolio.iloc[-1]
        else:
            continue

        annualized_return = (end_val_k_label_cumsum / start_val_k_label_cumsum) - 1
        ulcer_index = m_ulcer_index(val_portfolio)
        max_drawdown = abs(ffn.calc_max_drawdown(val_portfolio)) + 0.001
        calmar_ratio = annualized_return / max_drawdown

        val_best_k_labels_list.append(
            {
                "signal": signal,
                "k_label": k_label,
                "calmar_ratio": calmar_ratio,
                "ulcer_index": ulcer_index,
                "annualized_return": annualized_return,
                "max_drawdown": max_drawdown,
            }
        )
    return pd.DataFrame(val_best_k_labels_list)



splitter = SlidingWindowSplitter(
    window_length=train_size,
    fh=np.arange(1, val_test_horizon + 1),
    step_length=test_size,
)

return_df_list = []
for i, (train_idx, val_test_idx) in enumerate(splitter.split(df)):
    val_idx = val_test_idx[:test_size]
    test_idx = val_test_idx[test_size:]

    first_train_idx = train_idx[:first_train_size]
    second_train_idx = train_idx[first_train_size:]

    # df_train = df.iloc[train_idx, :]

    df_first_train = df.iloc[first_train_idx, :]
    df_second_train = df.iloc[second_train_idx, :]
    df_val = df.iloc[val_idx, :]
    df_test = df.iloc[test_idx, :]

    # FIRST TRAINING
    pips_first_train_df = get_pips_df(df_first_train)
    ts_scaler = StandardScaler().fit(
        pips_first_train_df[["day_of_week", "hour", "minute"]]
    )
    pips_first_train_df[["day_of_week", "hour", "minute"]] = ts_scaler.transform(
        pips_first_train_df[["day_of_week", "hour", "minute"]]
    )

    pips_second_train_df = get_pips_df(df_second_train)
    pips_second_train_df[["day_of_week", "hour", "minute"]] = ts_scaler.transform(
        pips_second_train_df[["day_of_week", "hour", "minute"]]
    )

    train_best_k_labels_df, kmeans = cluster_and_filter_pips_df(
        pips_first_train_df, pips_second_train_df
    )
    # ACCEPT OR REJECT THE TRAIN MODEL
    if train_best_k_labels_df.empty:
        continue

    # VALIDATION
    pips_val_df = get_pips_df(df_val)
    pips_val_df[["day_of_week", "hour", "minute"]] = ts_scaler.transform(
        pips_val_df[["day_of_week", "hour", "minute"]]
    )
    val_best_k_labels_df = filter_pips_df(pips_val_df, train_best_k_labels_df, kmeans)

    # ACCEPT OR REJECT THE VAL MODEL
    if val_best_k_labels_df.empty:
        continue
    
    if (
        val_best_k_labels_df["annualized_return"].sum() < 0
        # or val_best_k_labels_df["max_drawdown"].sum() > 0.1
        # or val_best_k_labels_df["calmar_ratio"].mean() < 2
    ):
        continue

    # TESTING
    pips_test_df = get_pips_df(df_test)
    pips_test_df[["day_of_week", "hour", "minute"]] = ts_scaler.transform(
        pips_test_df[["day_of_week", "hour", "minute"]]
    )
    test_best_k_labels_df = filter_pips_df(pips_test_df, val_best_k_labels_df, kmeans)

    # check if val_best_k_labels_df or test_best_k_labels_df is not empty
    if val_best_k_labels_df.empty or test_best_k_labels_df.empty:
        continue

    return_df_list.append(
        {
            "window": i,
            "train_sum_annualized_return": train_best_k_labels_df[
                "annualized_return"
            ].sum(),
            "val_sum_annualized_return": val_best_k_labels_df[
                "annualized_return"
            ].sum(),
            "val_sum_max_drawdown": val_best_k_labels_df["max_drawdown"].sum(),
            "val_avg_calmar_ratio": val_best_k_labels_df["calmar_ratio"].mean(),
            "test_sum_annualized_return": test_best_k_labels_df[
                "annualized_return"
            ].sum(),
            "test_sum_max_drawdown": test_best_k_labels_df["max_drawdown"].sum(),
            "test_avg_calmar_ratio": test_best_k_labels_df["calmar_ratio"].mean(),
        }
    )
    # print(f"Window: {i}")
    # print(test_best_k_labels_df)
    if i >= 5:
        break
return_df = pd.DataFrame(return_df_list)

In [ ]:
# (return_df["train_sum_annualized_return"]).cumsum().plot()
(return_df["test_sum_annualized_return"]).cumsum().plot()
(return_df['val_sum_annualized_return']).cumsum().plot()
# show legend
# (return_df['test_avg_calmar_ratio']).cumsum().plot()
# (return_df['val_avg_calmar_ratio']).cumsum().plot()
plt.legend()
# show graph
plt.show()

In [ ]:
return_df = pd.DataFrame(return_df_list)
return_df


In [ ]:
pips_first_train_df